In [1]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date


In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here



#initial load kc_table_id = "opendatadbt.311.kc311"

kc311 = requests.get("https://data.kcmo.org/resource/7at3-sxhp.json? limit=10000& where=creation_date >= '2021-03-17T00:00:00.000'" ) kc311json = kc311.json() kc_df = pd.DataFrame(kc311json) kc_df = kc_df[['creation_date', 'closed_date','department','work_group','request_type','zip_code','street_address','status','ycoordinate','xcoordinate']] kc_df

In [51]:
#Incremental load
nyc_table_id = "opendatadbt.311.kc311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
max(creation_date), max(closed_date)
FROM `opendatadbt.311.kc311`
limit 10
"""

kcmaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
kcmaxopendatedate = str(kcmaxopendatedataframe['f0_'][0])

kcmaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
kcmaxclosedate = str(kcmaxclosedatedataframe['f1_'][0])


In [52]:

kc311 = requests.get("https://data.kcmo.org/resource/7at3-sxhp.json?$limit=50000&$where=creation_date>'"+kcmaxopendatedate+"' OR closed_date between '" +kcmaxclosedate+ "' AND '" + current_datetime + "'") 
kc311json = kc311.json()


In [53]:
cols = ['case_id','creation_date', 'closed_date','department','work_group','request_type','zip_code','street_address','status','ycoordinate','xcoordinate']
kc_df = pd.DataFrame(kc311json, columns=cols)
kc_df

,case_id,creation_date,closed_date,department,work_group,request_type,zip_code,street_address,status,ycoordinate,xcoordinate
0,2020079604,2020-06-09T00:00:00.000,2021-03-29T00:00:00.000,NHS,NHS-Neighborhood Preservation-,Property Violations,64130,4430 Chestnut Ave,RESOL,39.045206,-94.552153
1,2021020802,2021-02-26T00:00:00.000,2021-03-29T00:00:00.000,Health,Health-Rat-,Animals / Pets-Rat Treatment-Inside Residence,64134,11717 Fremont Ave,RESOL,38.911211,-94.516323
2,2021014945,2021-02-16T00:00:00.000,2021-03-29T00:00:00.000,Northland,Northland-Neighborhood-Inc,Prop/Build/Construct-Property Maintenance-NNI,64116,120 NW Greentree Ln,RESOL,39.162877,-94.583542
3,2021023440,2021-03-11T00:00:00.000,2021-03-29T00:00:00.000,Northland,Northland-Neighborhood-Inc,Prop/Build/Construct-Property Maintenance-NNI,64119,4822 N Winchester Ave,RESOL,39.181839,-94.499293
4,2021022659,2021-03-03T00:00:00.000,2021-03-29T00:00:00.000,Northland,Northland-Neighborhood-Inc,Prop/Build/Construct-Property Maintenance-NNI,64118,5220 N Troost Ave,RESOL,39.189737,-94.567941
...,...,...,...,...,...,...,...,...,...,...,...
204,2021024384,2021-03-29T00:00:00.000,NaN,NHS,NHS-Neighborhood Preservation-,Property Violations,64130,5200 Prospect Ave,OPEN,39.031793,-94.555227
205,2021024385,2021-03-29T00:00:00.000,NaN,NHS,NHS-Neighborhood Preservation-,Property Violations,64130,5146 Prospect Ave,OPEN,39.032185,-94.555204
206,2021003145,2021-01-08T00:00:00.000,2021-03-29T00:00:00.000,Water Services,Water Services-Line Maintenance-Pipeline,Water-Pipeline Referral-Hydrant,64132,7803 Indiana Ave,RESOL,38.984636,-94.547872
207,2020156843,2020-12-18T00:00:00.000,2021-03-29T00:00:00.000,Northland,Northland-Neighborhood-Inc,Prop/Build/Construct-Property Maintenance-NNI,64119,5257 NE Munger Ave,RESOL,39.18943,-94.513821


In [54]:
kc_table_id = "opendatadbt.311.kc311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('kc311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    kc_df, table_ref, job_config=job_config
).result()  # Make an API request.


In [4]:
kc_table_id = "opendatadbt.311.kc311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
count(*), current_date(), "KC"
FROM `opendatadbt.311.kc311`

"""

kcdatacount = (
    client.query(query_string)
    .result()
    .to_dataframe()
)

kcdatacount = kcdatacount.rename(columns={'f0_':'count','f1_':'date','f2_':'city'})

kc_table_id = "opendatadbt.311.extractrowcount"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('extractrowcount')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",
)

client.load_table_from_dataframe(
    kcdatacount, table_ref, job_config=job_config
).result()  # Make an API request.